In [2]:
!pip install kagglehub



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [15]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("redwankarimsony/heart-disease-data")

print("Path to dataset files:", path)

import pandas as pd
df=pd.read_csv('heart_disease_uci.csv')
df


Path to dataset files: /Users/ellaforss/.cache/kagglehub/datasets/redwankarimsony/heart-disease-data/versions/6


,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,916,54,Female,VA Long Beach,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
916,917,62,Male,VA Long Beach,typical angina,NaN,139.0,False,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN,0
917,918,55,Male,VA Long Beach,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,919,58,Male,VA Long Beach,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# -------------------------------
# 1. Load dataset
file_path = "/Users/ellaforss/.cache/kagglehub/datasets/redwankarimsony/heart-disease-data/versions/6/heart_disease_uci.csv"
df = pd.read_csv(file_path)
print("Initial dataset shape:", df.shape)

# -------------------------------
# 2. Drop irrelevant columns
df = df.drop(['id','dataset'], axis=1)

# -------------------------------
# 3. Handle missing values
# Fill numeric columns with median
df.loc[df['trestbps'] == 0, 'trestbps'] = np.nan
df.loc[df['chol'] == 0, 'chol'] = np.nan

numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())



# Fill categorical columns with mode
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# -------------------------------
# 4. Encode categorical variables (one-hot encoding)
df = pd.get_dummies(df, drop_first=True)

# -------------------------------
# 5. Split features and target
X = df.drop('num', axis=1)
y = df['num']

# -------------------------------
# 6. Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# -------------------------------
# 7. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# -------------------------------
# 8. Handle class imbalance on training set using SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# -------------------------------
# 9. Final dataset info
print("Original training set shape:", X_train.shape, y_train.shape)
print("Resampled training set shape:", X_train_res.shape, y_train_res.shape)
print("Test set shape:", X_test.shape, y_test.shape)

# Optional: check target distribution after resampling
print(pd.Series(y_train_res).value_counts())


Initial dataset shape: (920, 16)
Original training set shape: (736, 18) (736,)
Resampled training set shape: (1645, 18) (1645,)
Test set shape: (184, 18) (184,)
num
3    329
0    329
2    329
1    329
4    329
Name: count, dtype: int64


In [21]:
X_train


array([[-0.05423496, -0.677285  , -0.11057068, ..., -0.5320941 ,
         0.59073988, -0.51355259],
       [ 2.17516813, -0.11924971, -0.11057068, ..., -0.5320941 ,
         0.59073988, -0.51355259],
       [-0.05423496, -0.11924971, -0.11057068, ..., -0.5320941 ,
        -1.69279243,  1.94722024],
       ...,
       [ 0.05192709, -0.45407088,  0.40143562, ..., -0.5320941 ,
        -1.69279243,  1.94722024],
       [ 0.4765753 ,  0.99682085,  0.72989249, ...,  1.87936684,
         0.59073988, -0.51355259],
       [-1.9651519 , -0.677285  ,  1.2129173 , ..., -0.5320941 ,
         0.59073988, -0.51355259]])